In [ ]:
import pandas as pd

In [ ]:
df_household = pd.read_excel('household_bmc.xlsx')
df_person = pd.read_excel('person_bmc.xlsx')
df_trip = pd.read_excel('trip_bmc.xlsx')
df_vehicle = pd.read_excel('vehicle_bmc.xlsx')
